In [1]:
import os
import cv2
import numpy as np
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score

# Directoarele pentru imaginile de antrenare și testare
train_folder = "./Faces/Train"
test_folder = "./Faces/Test"

# Funcție pentru încărcarea imaginilor și etichetelor
def load_images(folder):
    images = []
    labels = []
    for file_name in os.listdir(folder):
        if file_name.endswith(".jpg"):
            # Citește imaginea și o transformă în grayscale
            img = cv2.imread(os.path.join(folder, file_name), cv2.IMREAD_GRAYSCALE)
            images.append(img.flatten())  # Vectorizează imaginea
            labels.append(int(file_name.split('_')[0]))  # Extrage ID-ul persoanei din nume
    return np.array(images), np.array(labels)

# Încarcă imaginile de antrenare și testare
train_images, train_labels = load_images(train_folder)
test_images, test_labels = load_images(test_folder)

# Numărul de componente principale (PCA)
num_components = 10

# Antrenare: creează un model PCA pentru fiecare persoană
person_pca_models = {}
mean_faces = {}

for person_id in np.unique(train_labels):
    # Selectează imaginile persoanei curente
    person_images = train_images[train_labels == person_id]

    # Calculează fața medie
    mean_face = np.mean(person_images, axis=0)
    mean_faces[person_id] = mean_face

    # Centrează datele
    centered_data = person_images - mean_face

    # Aplică PCA
    pca = PCA(n_components=num_components)
    pca.fit(centered_data)
    person_pca_models[person_id] = pca

# Testare: clasificarea imaginilor de test
predictions = []

for i, test_img in enumerate(test_images):
    reconstruction_errors = []

    for person_id, pca in person_pca_models.items():
        # Centrează imaginea de test față de fața medie a persoanei
        centered_test_img = test_img - mean_faces[person_id]

        # Proiectează imaginea pe subspațiul PCA
        projected_img = pca.transform([centered_test_img])

        # Reconstruiește imaginea din subspațiu
        reconstructed_img = pca.inverse_transform(projected_img)

        # Calculează eroarea de reconstrucție (distanța euclidiană)
        error = np.linalg.norm(centered_test_img - reconstructed_img)
        reconstruction_errors.append((person_id, error))

    # Alege persoana cu cea mai mică eroare de reconstrucție
    predicted_person = min(reconstruction_errors, key=lambda x: x[1])[0]
    predictions.append(predicted_person)

    print(f"Test Image {i+1}: True Person {test_labels[i]}, Predicted Person {predicted_person}")

# Evaluare: calculează acuratețea clasificării
accuracy = accuracy_score(test_labels, predictions)
print(f"Classification Accuracy: {accuracy * 100:.2f}%")


Test Image 1: True Person 3, Predicted Person 3
Test Image 2: True Person 2, Predicted Person 2
Test Image 3: True Person 2, Predicted Person 2
Test Image 4: True Person 1, Predicted Person 1
Test Image 5: True Person 3, Predicted Person 3
Test Image 6: True Person 1, Predicted Person 1
Test Image 7: True Person 1, Predicted Person 1
Test Image 8: True Person 3, Predicted Person 3
Test Image 9: True Person 5, Predicted Person 5
Test Image 10: True Person 1, Predicted Person 1
Test Image 11: True Person 2, Predicted Person 2
Test Image 12: True Person 5, Predicted Person 5
Test Image 13: True Person 1, Predicted Person 1
Test Image 14: True Person 4, Predicted Person 4
Test Image 15: True Person 4, Predicted Person 4
Test Image 16: True Person 3, Predicted Person 3
Test Image 17: True Person 2, Predicted Person 2
Test Image 18: True Person 3, Predicted Person 3
Test Image 19: True Person 5, Predicted Person 5
Test Image 20: True Person 2, Predicted Person 2
Test Image 21: True Person 3,